In [3]:
import pandas as pd
from collections import defaultdict
import re
import logging
import os
from datetime import datetime

In [12]:
heute = datetime.now().strftime('%y-%m-%d')

if not os.path.isdir(f"../abzug/{heute}"):
    os.mkdir(f"../abzug/{heute}")

In [5]:
## allgemeine Funktionen

def make_nested_frame(dict) -> pd.DataFrame:
    """
    Die Funktion nimmt ein dreifach verschachteltes Dict und generiert daraus ein DataFrame mit dreifachem index
    """
    return pd.DataFrame.from_dict({(k1, k2, k3): v3 
      for k1, v1 in dict.items() 
      for k2, v2 in v1.items()
      for k3, v3 in v2.items()},
      orient='index', dtype='object')

def feldauswertung(kategorie: str, inhalt: str) -> list[tuple[str,str]]:
    """
    Die Funktion hat zwei Argumente: die Pica-Kategorie und den Feldinhalt.
    Per regex wird das gewünschte Unterfeld gesucht und in einem tuple mit der Klarfeldbezeichnung ausgegeben.
    Weil auch Unterfelder wiederholbar sind, kommt eine Liste von Tupeln zurück.
    """
    ergebnisse = list()
    matchlist = [
        ('209A','f','standort'),
        ('209A','g','signatur_g'),
        ('209A','d','ausleihcode'),
        ('209A','a','signatur_a'),
        ('209A','c','sig_komm'),
        ('209C','a','akz'),
        ('237A','a','f4801_a'),
        ('237A','k','f4801_k'),
        ('247C','9','bibliothek')]

    for match in matchlist:
        if kategorie == match[0]:        
            ergebnisse.append((match[2],'; '.join(re.findall(f"\${match[1]}([^\$]+)", inhalt))))
    
    return ergebnisse


def get_exemplare(dateiname: str) -> pd.DataFrame:
    """
    Auf IDN und Exemplardatenfelder reduzierter Dump, dessen Dateiname als Argument übergeben wird, wird in df mit gewünschten Feldern umgewandelt
    """
    with open(f'../raw/{dateiname}', 'r') as f:
        df = pd.DataFrame()
        for l in f:
            # print(l)
            line = l.split(' ',1)
            if l == '\n':
                df = pd.concat([df,make_nested_frame(exemplare)])

            elif line[0] == '003@':
                exemplare = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda:defaultdict(str))))
                idn = line[1][2:].strip()
            elif line[0] == '101@':
                einrichtung = line[1][2:].strip()
            
            else:
                feld = line[0].split('/')
                feldergebnis = feldauswertung(feld[0],line[1].strip())
                for ergebnis in feldergebnis:
                    # der seltsame join deshalb, weil Felder auch wiederholt werden können und die Ergebnisse dann zusammengezogen und durch Semikolon getrennt werden. wahrscheinlich nicht die beste lösung.
                    exemplare[idn][einrichtung][feld[1]][ergebnis[0]] = '; '.join([ergebnis[1],exemplare[idn][einrichtung][feld[1]][ergebnis[0]]])  
        
        df.index.set_names(['idn','einrichtung','exemplar'], inplace=True)
        return df
            



In [6]:
class Numeral:
    conc = {
        "i" : 1,
        "v" : 5,
        "x" : 10,
        "l" : 50,
        "c" : 100,
        "d" : 500,
        "m" : 1000
    }
    conca = {
        1000 : "m",
        500 : "d",
        100 : "c",
        50 : "l",
        10 : "x",
        5 : "v",
        1 : "i"
    }
    subst = {
        "dcccc" : "cm",
        "lxxxx" : "xc",
        "viiii" : "ix",
        "cccc" : "cd",
        "xxxx" : "xl",
        "iiii" : "iv"
    }

def to_arabic(lett):
    sum = 0
    subtr = False
    lett = lett.strip().lower().replace(".", "").replace(" ", "")
    if re.match(r"[ivxlcdm]+", lett) == None:
        return(None)
    rev = lett[::-1]
    last = 0
    for let in rev:
        val = Numeral.conc[let]
        if val >= last:
            sum += val
            subtr = False
        elif val < last:
            sum -= val
            subtr = True
        elif val == last and subtr == True:
            sum -= val
        last = val
    return(sum)
    
def to_roman(num):
    if isinstance(num, int) == False:
        return(None)
    rom = ""
    while num > 0:
        for key in Numeral.conca:
            if num >= key:
                rom = rom + Numeral.conca[key]
                num -= key
                break
    for add, sub in Numeral.subst.items():
        rom = rom.replace(add, sub)
    return(rom)

def get_norm_p(pages):
    normp = 0
    chunks = re.findall(r"(([^BS]+) (Bl)|([^BS]+) (S$|S[^p]|Bo)|([^BS]+) Sp)", pages)
    for ch in chunks:
        wh, numbl, _bl, nums, _sbo, numsp = ch
        if "-" in wh:
            continue
        if numbl != "":
            normp += get_number(numbl, 2)
        elif nums != "":
            normp += get_number(nums)
        elif numsp != "":
            normp += get_number(numsp, 0.5)
    chunks2 = re.findall(r"S\.? \d+ ?- ?\d+", pages)
    for ch2 in chunks2:
        normp += get_number(ch2)
    return(normp)

def get_number(page_string, mult=1):
    res = 0
    clean = re.sub(r"[\divxdclmIVXDCLM]+,? \[?(das heißt|i. ?e.)", "", page_string)
    spans = re.findall("(\[?(\d+)\]? ?- ?\[?(\d+)\]?)", clean)
    for span in spans:
        whole, start, end = span
        diff = int(end) - int(start)
        clean = re.sub(whole, str(diff), clean)
    extract = re.findall(r"\d+", clean)
    for num in extract:
        res += int(num)
    extract = re.findall(r"([ivxdclm]+) ", clean.lower())
    for num in extract:
        arab = to_arabic(num)
        if arab == None:
            logging.error(f"Nicht zu parsen: {num}")
        else:
            res += arab
    return(int(res * mult))

In [11]:
df = pd.read_csv("../abzug/böink.csv")
df["normpages"] = df.umfang.map(get_norm_p, na_action='ignore')

In [12]:
df

,idn,akz,bbg,standort,signatur_g,signatur_a,titel,stuecktitel,umfang,f4243,f4256,f4241,f4105_9,f4105_g,ausleihcode,f4801_a,f4801_k,einrichtung,exemplar,normpages
0,1072233045,L-1495-327139064,Aa,DBSM/M/Bö,Bö Ink 1,Bö Ink 1,Ars minor :,:,32 Bl.,NaN,NaN,NaN,NaN,NaN,NaN,"unvollst., Bl. 1, 2, 13 und 18 fehlen",NaN,1,2,64.0
1,106696517X,L-1488-315495324,Aa,DBSM/M/Bö,Bö Ink 2,Bö Ink 2,Flores astrologiae :,:,20 Blätter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,40.0
2,1066969973,L-1492-31550031X,Aaf,DBSM/M/Boe,Bö Ink 3,Bö Ink 3,Poeniteas cito :,:,18 Bl.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,36.0
3,1072233584,L-1498-32713948X,Aa,DBSM/M/Bö,Bö Ink 4,Bö Ink 4,Hymni. Expositio hymnorum :,:,50 Bl.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,100.0
4,1066970009,L-1478-315500344,Aa,DBSM/M/Bö,Bö Ink 5,Bö Ink 5,Sphaera mundi :,:,48 Bl.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,1108322905,L-1476-373399480,Aa,DBSM/M/Bö,Bö Ink 274 - Fragm.,Bö Ink 274 - Fragm.,De horis canonicis :,:,"28 ungezählte Blätter, 33-34 Zeilen",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,56.0
261,1066966710,L-1478-375750975,Aaf,DBSM/M/Bö,Bö Ink 275 - Fragm.,Bö Ink 275 - Fragm.,"Biblia, niederdt. :",:,544 Bl.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,1088.0
262,1113034386,L-9999-37967601X,Aa,DBSM/M/Bö,Bö Ink 277 - Fragm.,Bö Ink 277 - Fragm.,Evangelium :,:,14 ungezählte Blätter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,28.0
263,1079551522,L-1470-379676575,Aal,DBSM/M/Bö,Bö Ink 278 - Fragm.,Bö Ink 278 - Fragm.,Errores Iudaeorum ex Talmud extracti. [Daran:] Probationes Novi Testamenti ex Veteri Testamento :,:,12 Bl.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,24.0


In [ ]:
pica filter -s "(017A.a == 'ya' || 070A.c =^ 'Angeb' || 070A.c =^ 'angeb')" /home/andre/Projekte/vrekon/dump/dbsm-titel-exemplare.dat.gz | pica select "003@.0, 021A.a, 011@.a,209A/*{a?,a},036H{8,9}" -H "idn,titel,jahr,signatur,qd_titel,qd_idn" -o ya.csv

In [7]:
df = pd.read_csv('../ya.csv',dtype="string")
df.sort_values("jahr")

,idn,titel,jahr,signatur
1515,1066966338,Speculum de honestate vitae,1472,"II 4,4e"
1500,1036845559,Clarissimi Viri Hyginii Poeticon Astronomicon. Opvs Vtilissimum Foeliciter Incipit De mundi & sph[a]er[a]e ac utriusq[ue] p[ar]tiu[m] declaratione,1485,"II 82,7 h (angebunden)"
1518,1132093619,Suma Johannis An-||dree compendiosa et vtilis ordinata sup|| secundo decretaliu. tractans de processu|| Judiciario. atque iuridice se exercere vo||lentibus summe necessaria||,1500,"III 21, 39 (2. angebundenes Werk)"
1498,1023794667,Solinus de memorabilibus Mundi,1503,"III 73, 61"
91,99393689X,Suma Johannis An||dree Su#[pro] quarto dcretaliu ~q et si breuis ? ver||bis. vtilitate tamen lõga satis. quare alijs sum||mis prolixioribus iure venit ista #[et] cõmoditate,1505,"III 21, 39 (1. angebundenes Werk)"
...,...,...,...,...
1222,1001865162,Druckannalen der Fürstlichen Buchdruckerei zu Barth in Pommern (nach Oelrichs),19XX,Da 388
958,999871714,Die @alte märkische Papierindustrie,19XX,Ba 330
784,998777765,Dva trudnych slučaja vosstanovlenika ugasšego teksta,19XX,Fb 33
454,996368353,Gestehungskosten-Tabellen zur Errechnung des Preises einer Satz- und Druckstunde,19XX,Bö B III 1639


In [3]:
def blacklist():
    liste = list()
    with open("../blacklist.txt", "r") as f:
        for line in f:
            if line.startswith("#"):
                continue
            
            liste.append(line.split("#")[0].strip())

    return tuple(liste)

blacklist()

('1003856543',
 '1064781187',
 '997383259',
 '994726791',
 '997309466',
 '1066785597',
 '',
 '1066869103',
 '1100197079',
 '1132656737',
 '1132656745')